In [2]:
import pandas as pd
import glob
import os

# Define the path to the Downloads folder
downloads_folder = "/Users/shodges/scripts/genesisenergy/data"

# Define the file pattern to match
# Note: The double asterisk (**) is used to match any characters in the filename
file_pattern = os.path.join(downloads_folder, 'Genesis*.csv')

# Get a list of all matching CSV files
csv_files = glob.glob(file_pattern, recursive=True)

# Check if any files were found
if not csv_files:
    print(f"No CSV files found matching the pattern {file_pattern}")
    exit(1)
else:
    print(f"Found {len(csv_files)} CSV files.")
    
# Print the list of found files
#    for file in csv_files:
#        print(file)

# Read each CSV file into a DataFrame and store them in a list
# Initialize an empty list to store DataFrames
dataframes = []

# Loop through each CSV file and read it into a DataFrame
for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Print the combined DataFrame
print(combined_df)

# Optionally, save the combined DataFrame to a new CSV file
#combined_df.to_csv(os.path.join(downloads_folder, "combined_data.csv"), index=False)

import re

def remove_day_suffix(text):
  """Removes ordinal suffixes from a string."""
  return re.sub(r"(?<=\d)(st|nd|rd|th)", "", text)

# Apply the function to the column
combined_df['date1'] = combined_df['date'].apply(remove_day_suffix)

combined_df['datetime'] = pd.to_datetime(combined_df['date1'], format='%I:%M%p %d %B %Y')
combined_df = combined_df.sort_values(by='datetime')
combined_df['YYYYMMDD'] = combined_df['datetime'].dt.strftime('%Y-%m-%d')
combined_df['Weekday'] = pd.to_datetime(combined_df['datetime'], format='%a').dt.day_name()
combined_df['Month'] = pd.to_datetime(combined_df['datetime'], format='%b').dt.month_name()
combined_df['Year'] = pd.to_datetime(combined_df['datetime'], format='%Y').dt.year

# Set time of day based on your specified time frames
combined_df.loc[combined_df.set_index('datetime').index.indexer_between_time('18:00', '23:59'), 'd_time'] = 'Po'
combined_df.loc[combined_df.set_index('datetime').index.indexer_between_time('0:00', '5:59'), 'd_time'] = 'Atapo'
combined_df.loc[combined_df.set_index('datetime').index.indexer_between_time('6:00', '11:59'), 'd_time'] = 'Ata'
combined_df.loc[combined_df.set_index('datetime').index.indexer_between_time('12:00', '17:59'), 'd_time'] = 'Ahiahi'

combined_df.loc[combined_df.set_index('datetime').index.indexer_between_time('0:00', '3:59'), 'd_time1'] = 'Atapo'
combined_df.loc[combined_df.set_index('datetime').index.indexer_between_time('4:00', '7:59'), 'd_time1'] = 'Breakfast'
combined_df.loc[combined_df.set_index('datetime').index.indexer_between_time('8:00', '11:59'), 'd_time1'] = 'Ata'
combined_df.loc[combined_df.set_index('datetime').index.indexer_between_time('12:00', '15:59'), 'd_time1'] = 'Ahiahi'
combined_df.loc[combined_df.set_index('datetime').index.indexer_between_time('16:00', '19:59'), 'd_time1'] = 'Dinner'
combined_df.loc[combined_df.set_index('datetime').index.indexer_between_time('20:00', '23:59'), 'd_time1'] = 'Po'

combined_df[['usage','units']] = combined_df['usage'].str.split(' ',expand=True)
combined_df['usage'] = pd.to_numeric(combined_df['usage'], errors='coerce')
combined_df['dollars'] = combined_df['dollars'].str.replace('$','')
combined_df['dollars'] = pd.to_numeric(combined_df['dollars'], errors='coerce')


combined_df.to_csv('combined.csv', index=False)


Found 119 CSV files.
                           date     usage dollars    type
0     12:00AM 2nd February 2025  0.20 kWh   $0.11  actual
1     01:00AM 2nd February 2025  0.17 kWh   $0.10  actual
2     02:00AM 2nd February 2025  0.17 kWh   $0.10  actual
3     03:00AM 2nd February 2025  0.18 kWh   $0.11  actual
4     04:00AM 2nd February 2025  0.17 kWh   $0.10  actual
...                         ...       ...     ...     ...
2852     07:00PM 2nd April 2025  0.76 kWh   $0.31  actual
2853     08:00PM 2nd April 2025  0.73 kWh   $0.30  actual
2854     09:00PM 2nd April 2025  1.02 kWh   $0.40  actual
2855     10:00PM 2nd April 2025  0.86 kWh   $0.35  actual
2856     11:00PM 2nd April 2025  0.46 kWh   $0.20  actual

[2857 rows x 4 columns]


In [ ]:
import datetime
ICP_NUMBER = '0084942401PC5B0'
df = combined_df.copy()

df['ICP_NUMBER'] = ICP_NUMBER
df['ESIID'] = df['ICP_NUMBER']
df['USAGE_DATE'] = df['YYYYMMDD']
df['REVISION_DATE'] = datetime.date.today().strftime('%Y-%m-%d')
df['USAGE_START_TIME'] = df['datetime'].dt.strftime('%H:%M')
df['USAGE_END_TIME'] = (df['datetime']+ pd.Timedelta(hours=1)).dt.strftime('%H:%M')
df['USAGE_KWH'] = df['usage']
df['ESTIMATED_ACTUAL'] = 'A'
df['CONSUMPTION_SURPLUSGENERATION'] = 'Consumption'
df['CATEGORY'] = df['d_time1']

drop_columns = ['ICP_NUMBER','date', 'date1', 'datetime', 'YYYYMMDD', 'Weekday', 'Month', 'Year', 'd_time', 'd_time1', 'units','dollars', 'usage','type']
df.drop(columns=drop_columns, inplace=True)

df.to_csv('IntervalData.csv', index=False)
print(df)

                ESIID  USAGE_DATE REVISION_DATE USAGE_START_TIME  \
48    0084942401PC5B0  2025-01-15    2025-05-14            00:00   
49    0084942401PC5B0  2025-01-15    2025-05-14            01:00   
50    0084942401PC5B0  2025-01-15    2025-05-14            02:00   
51    0084942401PC5B0  2025-01-15    2025-05-14            03:00   
52    0084942401PC5B0  2025-01-15    2025-05-14            04:00   
...               ...         ...           ...              ...   
1988  0084942401PC5B0  2025-05-13    2025-05-14            19:00   
1989  0084942401PC5B0  2025-05-13    2025-05-14            20:00   
1990  0084942401PC5B0  2025-05-13    2025-05-14            21:00   
1991  0084942401PC5B0  2025-05-13    2025-05-14            22:00   
1992  0084942401PC5B0  2025-05-13    2025-05-14            23:00   

     USAGE_END_TIME  USAGE_KWH ESTIMATED_ACTUAL CONSUMPTION_SURPLUSGENERATION  
48            01:00       0.17                A                   Consumption  
49            02:00    

ESIID,USAGE_DATE,REVISION_DATE,USAGE_START_TIME,USAGE_END_TIME,USAGE_KWH,ESTIMATED_ACTUAL,CONSUMPTION_SURPLUSGENERATION
'10443720007382518,03/01/2022,03/02/2022 06:18:33,00:00,00:15,.145,A,Consumption
'10443720007382518,03/01/2022,03/02/2022 06:18:33,00:15,00:30,.188,A,Consumption
'10443720007382518,03/01/2022,03/02/2022 06:18:33,00:30,00:45,.125,A,Consumption
'10443720007382518,03/01/2022,03/02/2022 06:18:33,00:45,01:00,.146,A,Consumption


conda create --name tsplot-3.7 python=3.7
conda activate tsplot-3.7
conda install -c conda-forge dash
conda install pandas numpy


https://extranet.trc.govt.nz/getdata/boo.hts?service=Hilltop&request=GetData&Site=Patea%20at%20Stratford&Measurement=Air%20Temperature%20(Continuous)&TimeInterval=P4M

In [34]:
print(df.head())


                         date  usage  dollars    type  \
48  12:00AM 15th January 2025   0.17     0.10  actual   
49  01:00AM 15th January 2025   0.21     0.12  actual   
50  02:00AM 15th January 2025   0.21     0.12  actual   
51  03:00AM 15th January 2025   0.19     0.11  actual   
52  04:00AM 15th January 2025   0.19     0.11  actual   

                      date1            datetime    YYYYMMDD    Weekday  \
48  12:00AM 15 January 2025 2025-01-15 00:00:00  2025-01-15  Wednesday   
49  01:00AM 15 January 2025 2025-01-15 01:00:00  2025-01-15  Wednesday   
50  02:00AM 15 January 2025 2025-01-15 02:00:00  2025-01-15  Wednesday   
51  03:00AM 15 January 2025 2025-01-15 03:00:00  2025-01-15  Wednesday   
52  04:00AM 15 January 2025 2025-01-15 04:00:00  2025-01-15  Wednesday   

      Month  Year  ... units       ICP_NUMBER            ESIID  USAGE_DATE  \
48  January  2025  ...   kWh  0084942401PC5B0  0084942401PC5B0  2025-01-15   
49  January  2025  ...   kWh  0084942401PC5B0  008494240

In [4]:
import plotly.graph_objects as go
import pandas as pd

df = combined_df.copy()
df['Category'] = df['d_time1']

# Create a bar chart
# Aggregate to monthly totals
df_monthly = df.groupby(['Month', 'Category'])['usage'].mean().reset_index()

ordered_months = ["January", "February", "March", "April", "May", "June",
      "July", "August", "September", "October", "November", "December"]

# sorting data accoring to ordered_months
df_monthly['to_sort']=df_monthly['Month'].apply(lambda x:ordered_months.index(x))
df_monthly = df_monthly.sort_values('to_sort')

ordered_dayparts = ["Atapo", "Breakfast", "Ata", "Ahiahi", "Dinner", "Po"]

fig = go.Figure()
for category in ordered_dayparts:
    df_category = df_monthly[df_monthly['Category'] == category]
    fig.add_trace(go.Bar(x=df_category['Month'], y=df_category['usage'], name=category))

fig.update_layout(
    title='Monthly Totals by Category',
    xaxis_title='Month',
    yaxis_title='Mean Value',
    barmode='group' # Display bars side by side
)

fig.show()

In [ ]:
from hilltoppy import Hilltop, utils

base_url = 'https://extranet.trc.govt.nz/getdata/'
hts = 'boo.hts'
site = ['Patea at Stratford']


# Create a Hilltop object
ht = Hilltop(base_url, hts)
sites_out1 = ht.get_site_list()
sites_out1.head()


,SiteName
0,0026 at Mangatete Stream
1,0047 at Tangahoe River
2,0047 at Tawhiti Stream
3,0124 at Waimoku Trib
4,0146 at Kapuni Stream


In [3]:
site_data = ht.get_site_info(site)
site_data

,SiteName,Altitude,Comment,ConsentCompliance,District,Easting,FirstSynonym,GISTag,GroundwaterZone,HIRDS,...,LawaSiteID,MetNumber,Northing,RecordingAuthority1,RecordingAuthority2,RiverNumber,SecondSynonym,ShortName,SiteID,SurfaceWaterZone
0,Patea at Stratford,295,TIDEDA site number - 943203 (R/F) - 843203 (Wi...,false,\n,1711572,\n,\n,Volcanics,\n,...,TRC-00037,E94323,5644489,TRC,\n,343000,\n,943203,32,Patea


In [5]:

site_meas = ht.get_measurement_list(site)
site_meas

,SiteName,MeasurementName,Units,Precision,Item,DataSourceName,TSType,DataType,Interpolation,From,To,VMStart,VMFinish,MeasurementGroup
0,Patea at Stratford,Rainfall,mm,1,1,Rainfall,StdSeries,SimpleTimeSeries,Incremental,2024-06-27 10:01:00,2025-05-15 20:31:00,NaT,NaT,NaN
1,Patea at Stratford,Rainfall-Difference,NaN,2,1,Rainfall,StdSeries,SimpleTimeSeries,Incremental,2024-06-27 10:01:00,2025-05-15 20:31:00,2024-06-27 10:01:00,2025-05-15 20:31:00,NaN
2,Patea at Stratford,RF TimeStep_60,NaN,2,1,Rainfall,StdSeries,SimpleTimeSeries,Incremental,2024-06-27 10:01:00,2025-05-15 20:31:00,2024-06-27 10:01:00,2025-05-15 20:31:00,VirtualChecks
3,Patea at Stratford,Voltage,V,2,1,Voltage,StdSeries,SimpleTimeSeries,Instant,2024-06-27 10:30:00,2025-05-15 20:30:00,NaT,NaT,NaN
4,Patea at Stratford,Air Temperature (Continuous),oC,2,1,Air Temperature (Continuous),StdSeries,SimpleTimeSeries,Instant,2024-06-27 10:10:00,2025-05-15 20:30:00,NaT,NaT,NaN
5,Patea at Stratford,AT TimeStep_600,NaN,2,1,Air Temperature (Continuous),StdSeries,SimpleTimeSeries,Instant,2024-06-27 10:10:00,2025-05-15 20:30:00,2024-06-27 10:10:00,2025-05-15 20:30:00,VirtualChecks
6,Patea at Stratford,Air Temp Diff CC,C,2,1,Air Temperature (Continuous),StdSeries,SimpleTimeSeries,Instant,2024-06-27 10:10:00,2025-05-15 20:30:00,2024-06-27 10:10:00,2025-05-15 20:30:00,Control Charts
7,Patea at Stratford,AirT-7day,deg.C,2,1,Air Temperature (Continuous),StdSeries,SimpleTimeSeries,Instant,2024-06-27 10:10:00,2025-05-15 20:30:00,2024-06-30 22:10:00,2025-05-15 20:30:00,NaN
8,Patea at Stratford,Wind Direction,deg,0,1,Wind Direction,StdSeries,SimpleTimeSeries,Instant,2024-06-27 10:10:00,2025-05-15 20:30:00,NaT,NaT,NaN
9,Patea at Stratford,WD-smoothed,NaN,2,1,Wind Direction,StdSeries,SimpleTimeSeries,Instant,2024-06-27 10:10:00,2025-05-15 20:30:00,2024-06-27 10:10:00,2025-05-15 20:30:00,VirtualChecks


In [ ]:
from hilltoppy import Hilltop, utils
base_url = 'https://extranet.trc.govt.nz/getdata/'
hts = 'boo.hts'
site = 'Patea at Stratford'
measurement = 'Air Temperature (Continuous)'
from_date = '2025-05-01'
to_date = '2025-05-15'
ht = Hilltop(base_url, hts)
tsdata = ht.get_data(site, measurement, from_date=from_date, to_date=to_date)

,SiteName,MeasurementName,Time,Value
0,Patea at Stratford,Air Temperature (Continuous),2025-05-01 00:00:00,16.854641
1,Patea at Stratford,Air Temperature (Continuous),2025-05-01 00:10:00,16.795223
2,Patea at Stratford,Air Temperature (Continuous),2025-05-01 00:20:00,16.731232
3,Patea at Stratford,Air Temperature (Continuous),2025-05-01 00:30:00,16.740372
4,Patea at Stratford,Air Temperature (Continuous),2025-05-01 00:40:00,16.731232


In [13]:
import plotly.express as px

fig = px.line(tsdata, x='Time', y='Value', title='Air Temperature Over Time')
fig.update_xaxes(title_text='Time')
fig.update_yaxes(title_text='Temperature (°C)')
fig.update_layout(
    font_family='Arial',
    font_size=12,
    title_font_size=16
)
fig.show()



In [6]:
DOWNLOADS_FOLDER = "/Users/shodges/scripts/genesisenergy/data"
import pandas as pd
import os

def check_forecast_electricty_data():
    """
    Load the forecast.csv file and return the latest daily (last 7), weekly, and monthly forecast values.
    Dates are returned as ISO 8601 strings.

    Args:
        data_dir (str or Path): Directory where forecast.csv is stored.

    Returns:
        dict: {
            'daily': List of dicts with keys 'date', 'value', 'label',
            'weekly': Dict with keys 'date', 'value', 'label',
            'monthly': Dict with keys 'date', 'value', 'label', 'range'
        }
    """
    
    print(" + Checking forecast data...")

    
    file_path = DOWNLOADS_FOLDER + "/forecasts.csv"
    df = pd.read_csv(file_path, parse_dates=["date"], date_format="%Y-%m-%d")

    # Filter relevant durations
    df = df[df["duration"].isin(["day", "week", "month"])]

    result = {}

    # Last 7 daily forecasts
    daily_df = df[df["duration"] == "day"].sort_values("date", ascending=False).head(7).sort_values("date")
    result["daily"] = [
        {
            "date": row["date"].date().isoformat(),
            "value": row["value"],
            "label": row["forecast"]
        }
        for _, row in daily_df.iterrows()
    ]

    # Latest weekly forecast
    weekly_df = df[df["duration"] == "week"]
    if not weekly_df.empty:
        latest_week = weekly_df.sort_values("date", ascending=False).iloc[0]
        result["weekly"] = {
            "date": latest_week["date"].date().isoformat(),
            "value": latest_week["value"],
            "label": latest_week["forecast"]
        }

    # Latest monthly forecast
    monthly_df = df[df["duration"] == "month"]
    if not monthly_df.empty:
        latest_month = monthly_df.sort_values("date", ascending=False).iloc[0]
        result["monthly"] = {
            "date": latest_month["date"].date().isoformat(),
            "value": latest_month["value"],
            "label": latest_month["forecast"],
            "range": latest_month.get("label", "")
        }

    return result

test = check_forecast_electricty_data()
print(test)

 + Checking forecast data...
{'daily': [{'date': '2025-05-26', 'value': 5.09, 'label': 'daily cost'}, {'date': '2025-05-27', 'value': 5.27, 'label': 'daily cost'}, {'date': '2025-05-28', 'value': 5.64, 'label': 'daily cost'}, {'date': '2025-05-29', 'value': 5.49, 'label': 'daily cost'}, {'date': '2025-05-30', 'value': 5.39, 'label': 'daily cost'}, {'date': '2025-05-31', 'value': 5.37, 'label': 'daily cost'}, {'date': '2025-06-01', 'value': 5.98, 'label': 'daily cost'}], 'weekly': {'date': '2025-05-26', 'value': np.float64(38.23), 'label': 'Estimated weekly cost'}, 'monthly': {'date': '2025-05-30', 'value': np.float64(260.35), 'label': 'Estimated monthly bill', 'range': '12 May to 10 Jun'}}
